### This is a notebook to train a Bayesian neural network with Monte Carlo drop out.

Bayesian neural networks can quantify uncertainties in their prediction. In this notebook a Bayesian neural network is trained with drop out to learn the function $$ y = 0.4 \sin(4x) + 0.5 \cos(12x) $$
A zero mean Gaussian random noise with standard deviation $\sigma = 0.05$ is added to the train data. Both the aleatoric (noise in the data) and the epistemic (model uncertainty) uncertaintes are quantified.

This notebook uses [UQpy](https://uqpyproject.readthedocs.io/en/latest/) which is a general purpose python toolbox for uncertainty quantification. In the following cell we import the necessary libraries to set up the environment to run the code.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# UQpy imports
from UQpy.scientific_machine_learning.layers import Dropout
from UQpy.scientific_machine_learning.neural_networks import FeedForwardNeuralNetwork
from UQpy.scientific_machine_learning.trainers import Trainer
import logging

logger = logging.getLogger("UQpy")
logger.setLevel(logging.INFO)

torch.manual_seed(0);

Once the libraries are loaded, we build the neural network. The neural network consists of 3 hidden layers with ReLu activation function.

In [2]:
width = 20
network = nn.Sequential(
    nn.Linear(1, width),
    nn.ReLU(),
    nn.Linear(width, width),
    nn.ReLU(),
    nn.Linear(width, width),
    nn.ReLU(),
    nn.Linear(width, width),
    Dropout(drop_rate=0.1),
    nn.ReLU(),
    nn.Linear(width, 1),
)
model = FeedForwardNeuralNetwork(network)

Now, we define the dataset and the data loader. The dataset class takes the number of samples and the standard deviation of the noise parameter as the input to produce x,y pairs of training data. This class is given as an input to the pytorch DataLoader. 

In [3]:
class SinusoidalDataset(Dataset):
    def __init__(self, n_samples=20, noise_std=0.05):
        self.n_samples = n_samples
        self.noise_std = noise_std
        self.x = torch.cat((torch.linspace(-1, -0.2, int(n_samples/2)),torch.linspace(0.2, 1, int(n_samples/2)))).reshape(-1, 1)
        self.y = torch.tensor(0.4 * np.sin(4 * self.x) + 0.5 * np.cos(12 * self.x) +
                              np.random.normal(0, self.noise_std, self.x.shape), dtype=torch.float)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, item):
        return self.x[item], self.y[item]

train_data = DataLoader(SinusoidalDataset(), batch_size=20, shuffle=True)

We define the optimizer with the parameters such as the learning rate and call the Trainer implemented in UQpy to learn the parameters of the model.

In [4]:
model.drop(False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
trainer = Trainer(model, optimizer)

Here, we train the model upto a user specified epoch. The following cell can be changed from "Raw" to "Code" to run it.

In [ ]:
print("Starting Training...", end="")
trainer.run(train_data=train_data, epochs=5000)
print("done")
train_loss = trainer.history["train_loss"].detach().numpy()

[INFO] - 2024-05-20 12:46:44,082 - UQpy: Scientific Machine Learning: Beginning training FeedForwardNeuralNetwork
[INFO] - 2024-05-20 12:46:44,093 - UQpy: Scientific Machine Learning: Epoch 1 / 5,000 Train Loss 0.21573162078857422
[INFO] - 2024-05-20 12:46:44,094 - UQpy: Scientific Machine Learning: Epoch 2 / 5,000 Train Loss 0.21563641726970673
[INFO] - 2024-05-20 12:46:44,095 - UQpy: Scientific Machine Learning: Epoch 3 / 5,000 Train Loss 0.21554191410541534
[INFO] - 2024-05-20 12:46:44,096 - UQpy: Scientific Machine Learning: Epoch 4 / 5,000 Train Loss 0.2154478132724762
[INFO] - 2024-05-20 12:46:44,097 - UQpy: Scientific Machine Learning: Epoch 5 / 5,000 Train Loss 0.21535348892211914
[INFO] - 2024-05-20 12:46:44,098 - UQpy: Scientific Machine Learning: Epoch 6 / 5,000 Train Loss 0.21526165306568146
[INFO] - 2024-05-20 12:46:44,098 - UQpy: Scientific Machine Learning: Epoch 7 / 5,000 Train Loss 0.21517367660999298
[INFO] - 2024-05-20 12:46:44,099 - UQpy: Scientific Machine Learning

Starting Training...

[INFO] - 2024-05-20 12:46:44,283 - UQpy: Scientific Machine Learning: Epoch 227 / 5,000 Train Loss 0.20751874148845673
[INFO] - 2024-05-20 12:46:44,284 - UQpy: Scientific Machine Learning: Epoch 228 / 5,000 Train Loss 0.2074820101261139
[INFO] - 2024-05-20 12:46:44,284 - UQpy: Scientific Machine Learning: Epoch 229 / 5,000 Train Loss 0.20744717121124268
[INFO] - 2024-05-20 12:46:44,285 - UQpy: Scientific Machine Learning: Epoch 230 / 5,000 Train Loss 0.20741203427314758
[INFO] - 2024-05-20 12:46:44,286 - UQpy: Scientific Machine Learning: Epoch 231 / 5,000 Train Loss 0.20737504959106445
[INFO] - 2024-05-20 12:46:44,287 - UQpy: Scientific Machine Learning: Epoch 232 / 5,000 Train Loss 0.2073393166065216
[INFO] - 2024-05-20 12:46:44,288 - UQpy: Scientific Machine Learning: Epoch 233 / 5,000 Train Loss 0.2073032408952713
[INFO] - 2024-05-20 12:46:44,288 - UQpy: Scientific Machine Learning: Epoch 234 / 5,000 Train Loss 0.20726700127124786
[INFO] - 2024-05-20 12:46:44,289 - UQpy: Scientific

We perdict the outputs for the test examples using the trained model.

In [ ]:
checkpt = {'model':model.state_dict(),
            'Train_loss':train_loss,    
            }
torch.save(checkpt, 'MCD_results.pt')

In [ ]:
# Plotting Results
x = SinusoidalDataset().x
y = SinusoidalDataset().y
x_val = torch.linspace(-1, 1, 1000).view(-1,1)
y_val = (0.4 * torch.sin(4 * x_val) + 0.5 * torch.cos(12 * x_val))
pred_val = model(x_val)

We visualize the predictions using the following plots.

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(
    x.detach().numpy(),
    y.detach().numpy(), '*',
    label="Data", color='crimson', markersize=10
)
ax.plot(
    x_val.detach().numpy(),
    pred_val.detach().numpy(),
    label="Final Prediction",
    color='navy', linewidth=2
)
ax.plot(
    x_val.detach(), y_val.detach(),
    label="Exact",
    color="black",
    linestyle="dashed",
    linewidth=2
)

# Customize the plot
ax.set_title("Prediction", fontsize=16, fontweight='bold')
ax.set_xlabel("x", fontsize=14)
ax.set_ylabel("f(x)", fontsize=14)
ax.legend(loc='upper right', fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Improve the layout
plt.tight_layout()

# Show the plot
plt.show()

Convergence of the loss function is shown in the following.

In [ ]:
train_loss = trainer.history["train_loss"].detach().numpy()

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(train_loss)


# Customize the plot
ax.set_title("MCD Training Loss", fontsize=16, fontweight='bold')
ax.set_xlabel("Epoch", fontsize=14)
ax.set_ylabel("Loss", fontsize=14)
ax.legend(loc='upper right', fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Improve the layout
plt.tight_layout()
plt.show()

We perform uncertainty quantification by randomly dropping neurons and predicting the output. The variance in the prediction gives us measure of epistemic uncertianty. The total uncertainty 
$$ \sigma_{total}^2 = \sigma_{epistemic}^2 + \sigma_{aleatoric}^2 $$

In [ ]:
# %%
model.train()
model.drop(True) # this activates the dropout layers
n = 1000
samples = torch.zeros(len(x_val), n)
for i in range(n):
    samples[:, i] = model(x_val).detach().squeeze()
variance = torch.var(samples, dim=1)
standard_deviation = torch.sqrt(variance)

#%%
x_plot = x.squeeze().detach().numpy()
mu =  torch.mean(samples, dim=1)
sigma = standard_deviation.squeeze().detach().numpy()
total_sigma = np.sqrt(sigma**2 + 0.05**2)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the mean prediction
ax.plot(x_val, mu, label="Mean Prediction", color='navy', linewidth=2)

# Plot the data points
ax.plot(
    x.detach().numpy(),
    y.detach().numpy(), '*',
    label="Data", color='crimson', markersize=10
)

# Fill the uncertainties
ax.fill_between(
    x_val.view(-1).detach(), mu - (3 * sigma), mu + (3 * sigma), 
    label="Epistemic Uncertainty", color='skyblue', alpha=0.6
)
ax.fill_between(
    x_val.view(-1).detach(), mu - (3 * total_sigma), mu + (3 * total_sigma), 
    label="Total Uncertainty", color='lightcoral', alpha=0.3
)

# Customize the plot
ax.set_title("Monte Carlo Dropout $\mu \pm 3\sigma$", fontsize=16, fontweight='bold')
ax.set_xlabel("x", fontsize=14)
ax.set_ylabel("f(x)", fontsize=14)
ax.legend(loc='upper right', fontsize=12)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Improve the layout
plt.tight_layout()

# Show the plot
plt.show()
